# Automated ML

Import Dependencies. In the cell below, import all the dependencies that I will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
import os
import joblib
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
import requests
import json
from azureml.core.conda_dependencies import CondaDependencies
import sklearn
from azureml.core.model import Model

## Dataset

### Overview

I use the Heart-Failure Dataset from Kaggle and the task is to predict the mortality of the patients based on their clinical records. I create the workspace and experiment in the Azure ML studio. I check if the computer target already exists. If it does not then it creates one. Finally, I display the first rows of the Heart-Failure Dataset.

In [3]:
#Create a workspace and an experiment in Azureml
ws = Workspace.from_config()
experiment_name = 'Heart-Failure-AutoMlProject'
project_folder = './Heart-Failure-project'

experiment=Experiment(ws, experiment_name)

#Check if the cluster exists if it does not then create one.
# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-11-14T13:10:20.139000+00:00', 'errors': None, 'creationTime': '2020-11-14T13:10:15.968229+00:00', 'modifiedTime': '2020-11-14T13:10:32.461273+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS13_V2'}


In [4]:
# Get Heart Failure Dataset and add it in pandas
ds = Dataset.get_by_name(ws, name='Heart-Failure')

In [6]:
#Review the first 5 rows in the dataset
ds.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration
It is used an AutoML for the classification problem using the Heart Failure Dataset. The AutoML run will look for the best model in relation to the accuracy. The target feature is the "DEATH_EVENT". The experiment timeout is 30 mins and the maximum number of iterations which are executed in parallel are 5. 

In [7]:
# AutoML settings
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# AutoML config
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=ds,
                             label_column_name="DEATH_EVENT",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
# Submit experiment
Automl_run = experiment.submit(automl_config)

Running on remote.


## Run Details

`RunDetails` widget is used to show the different experiments.

In [9]:
RunDetails(Automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

Getb the best model from the automl experiments and display all the properties of the model.



In [41]:
best_run, fitted_model = Automl_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: Heart-Failure-AutoMlProject,
Id: AutoML_145744c5-84cf-4edf-9ef7-aee6ffbfe704_38,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                subsample_for_bin=200000,
                                                                                                subsample_freq=0,
               

In [42]:
#TODO: Save the best model
joblib.dump(fitted_model, 'automlmodel.pkl')

automl_model = Model.register(workspace=ws,
                       model_name='auto-ml',                # Name of the registered model in your workspace.
                       model_path='./automlmodel.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version=sklearn.__version__,# Version of scikit-learn used to create the model.
                      # resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=1),
                       description='Regression model to predict Death Event',
                       tags={'area': 'Death Event', 'type': 'regression'}
                       )

print('Name:', automl_model.name)
print('Version:', automl_model.version)

Registering model auto-ml
Name: auto-ml
Version: 4


## Model Deployment 1


Remember you have to deploy only one of the two models you trained. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [121]:
#Set up the Environment 
env = Environment(workspace=ws, name="myenv")  

#Install the correct packages
#env.python.conda_dependencies.add_pip_package("scikit-learn")  

# Define the packages needed by the model and scripts
#env.python.conda_dependencies.add_pip_package('ference-schema[numpy-support]')
env.python.conda_dependencies.add_pip_package("numpy")
env.python.conda_dependencies.add_pip_package("scikit-learn")
env.python.conda_dependencies.add_pip_package("azureml-train-automl")
env.python.conda_dependencies.add_pip_package("azureml-explain-model")
env.python.conda_dependencies.add_pip_package("inference-schema")
env.python.conda_dependencies.add_pip_package("py-xgboost")
env.python.conda_dependencies.add_pip_package("joblib")
# You must list azureml-defaults as a pip dependency
env.python.conda_dependencies.add_pip_package("azureml-defaults")
env.python.conda_dependencies.add_pip_package("conda")

In [114]:
#Set up the inference_config
inference_config = InferenceConfig(entry_script='score.py', environment=env)
#Set up the deployment_config as webservice
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

#Deploy the model
service = Model.deploy(
    workspace = ws,
    name = "mywebservice2",
    models = [automl_model],
    inference_config = inference_config,
    deployment_config = aci_config, overwrite=True)

#wait until deployment is complete
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................................................................
Failed


ERROR - Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: d6e193c1-f8cf-4150-86ad-5267241dccac
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: mywebservice2. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image 017e24476d9f42f498c63586fb784fde.azurecr.io/azureml/azureml_13566eba4a2d7f1d17c875fa7545ed0f locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for y

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: d6e193c1-f8cf-4150-86ad-5267241dccac
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: mywebservice2. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image 017e24476d9f42f498c63586fb784fde.azurecr.io/azureml/azureml_13566eba4a2d7f1d17c875fa7545ed0f locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's i

In [115]:
print(service.get_logs())

2020-11-14T16:40:39.9156389Z stdout F 2020-11-14T16:40:39,896504300+00:00 - gunicorn/run 
2020-11-14T16:40:39.9226233Z stdout F 2020-11-14T16:40:39,904211400+00:00 - rsyslog/run 
2020-11-14T16:40:39.9373732Z stdout F 2020-11-14T16:40:39,912266100+00:00 - iot-server/run 
2020-11-14T16:40:39.9374975Z stdout F 2020-11-14T16:40:39,911051600+00:00 - nginx/run 
2020-11-14T16:40:39.944035Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_3a737fba763fc576399cf0fee1b7331a/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-11-14T16:40:39.9466149Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_3a737fba763fc576399cf0fee1b7331a/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-11-14T16:40:39.9526158Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_3a737fba763fc576399cf0fee1b7331a/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-11-14T16:40:39.9526158Z stderr F /usr/sbin/nginx: 

In [ ]:
#Print the state
print(service.state)

#Print the scoring uri of the service
print(service.scoring_uri)

#Print the primary authentication key for the service
primary, secondary = service.get_keys()
print(primary)

In [ ]:
#Local Deployment
from azureml.core.webservice import LocalWebservice
local_config = LocalWebservice.deploy_configuration(port=9000)
inference_config = InferenceConfig(entry_script='score.py', environment=env)
local_service = Model.deploy(ws, "test", [automl_model], inference_config, local_config)
local_service.wait_for_deployment(show_output=True)


Generating Docker build context.
2020/11/14 16:49:39 Downloading source code...
2020/11/14 16:49:41 Finished downloading source code
2020/11/14 16:49:42 Creating Docker network: acb_default_network, driver: 'bridge'
2020/11/14 16:49:42 Successfully set up Docker network: acb_default_network
2020/11/14 16:49:42 Setting up Docker configuration...
2020/11/14 16:49:43 Successfully set up Docker configuration
2020/11/14 16:49:43 Logging in to registry: 017e24476d9f42f498c63586fb784fde.azurecr.io
2020/11/14 16:49:44 Successfully logged into 017e24476d9f42f498c63586fb784fde.azurecr.io
2020/11/14 16:49:44 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/11/14 16:49:44 Scanning for dependencies...
2020/11/14 16:49:44 Successfully scanned dependencies
2020/11/14 16:49:44 Launching container with name: acb_step_0
Sending build context to Docker daemon     64kB
Step 1/15 : FROM mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:2020082

In [83]:
print(local_service.get_logs())

2020-11-14T15:38:01,564252436+00:00 - gunicorn/run 
2020-11-14T15:38:01,566125851+00:00 - rsyslog/run 
2020-11-14T15:38:01,566883758+00:00 - iot-server/run 
2020-11-14T15:38:01,569551579+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_8fd3f9f199f53e0d69da685b735ef9f1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8fd3f9f199f53e0d69da685b735ef9f1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8fd3f9f199f53e0d69da685b735ef9f1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8fd3f9f199f53e0d69da685b735ef9f1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8fd3f9f199f53e0d69da685b735ef9f1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

TypeError: expected str, bytes or os.PathLike object, not NoneType

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# URL for the web service
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

# Two sets of data to score, so we get two results back
data = {"data":
        [
            [
            "age": 17
            "anaemia": 1
            "creatinine_phosphokinase": 452
            "diabetes": 0
            "ejection_fraction": 30
            "high_blood_pressure": 1
            "platelets": 263358.03
            "serum_creatinine": 1.1
            "serum_sodium": 137
            "sex": 0
            "smoking": 0
            "time": 8
            ],
        }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()

from azureml.core.model import Model
import logging
logging.basicConfig(level=logging.DEBUG)
print(Model.get_model_path(model_name='my-best-model'))